# Term Assignment Notebook

In [13]:
# Imports
import random
import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict, Set
from enum import Enum
import matplotlib.pyplot as plt
import numpy as np

# Load .env vars
load_dotenv()

# Connect to openai
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

#### Agents

In [2]:
class EggplaceUser():
    """
    Base definition of the user agent.
    """
    user_id: int
    username: str
    
    _instance_count = 0
    
    def __init__(self, user_id: int, username: str):
        self.user_id = user_id
        self.username = username
        EggplaceUser._instance_count += 1
    
    def __str__(self) -> str:
        return f"UserID: {self.user_id}, Username: {self.username}"
        
    @classmethod
    def get_instance_count(cls):
        return cls._instance_count

In [83]:
class ActivityLevel(Enum):
    """
    Enum to store information relevant to activity level.
    """
    PASSIVE = 0
    MODERATE = 1
    ACTIVE = 2

    def post_likelihood(self) -> float:
        """
        Reference likelihood to post based on user's activity level.
        """
        match self.name:
            case 'ACTIVE':
                return 0.4
            case _: 
                return 0.0
    
    def post_read_likelihood(self) -> float:
        """
        Reference likelihood to post based on user's activity level.
        """
        match self.name:
            case 'ACTIVE':
                return 1.0
            case 'MODERATE':
                return 0.8
            case 'PASSIVE':
                return 0.5
    
    def passion_range(self) -> float:
        match self.name:
            case 'ACTIVE':
                return random.uniform(-3.0,3.0)
            case 'MODERATE':
                return random.uniform(-1.5,1.5)
            case 'PASSIVE':
                return random.uniform(-1,1)

class CommonUser(EggplaceUser):
    """
    Voting users in eggplace.
    Three types:
    - Active user: can like posts and make posts, views all posts
    - Moderate user: can like posts, views many posts
    - Passive user: can like posts, views the least posts
    """
    user_id: int
    username: str
    usage_type: ActivityLevel
    likelihood_to_post: float
    vote_leaning: float
    preferences: Dict[str, float | dict]
    campaign_period: int

    def __init__(self, user_id:int, username:str, usage: ActivityLevel, max_days: int):
        super().__init__(user_id, username)
        self.usage_type = usage
        self.likelihood_to_post = usage.post_likelihood()
        self.vote_leaning = 0.5
        self.preferences = self.generate_preferences()
        self.campaign_period = max_days

    def __str__(self) -> str:
        return "Common(" + super().__str__() + f", Usage: {self.usage_type})"
    
    def __repr__(self):
        return self.__str__()
    
    def generate_preferences(self) -> Dict[str, float|dict]:
        """
        Generate dictionary of user preferences
        """
        budget = 2
        ethos_pref = random.uniform(0,2)
        pathos_pref = random.uniform(0, (budget - ethos_pref))
        logos_pref = budget - (ethos_pref + pathos_pref)

        preference_dict = {
            "ethos": ethos_pref,
            "pathos": pathos_pref,
            "logos": logos_pref,
            "tone_preference": {
                "angry": self.usage_type.passion_range(),
                "compasionate": self.usage_type.passion_range(),
                "funny": self.usage_type.passion_range(),
                "empassioned":self.usage_type.passion_range()
            }
        }
        return preference_dict
    
    def interpret_post(self, post_characteristics: Dict[str, int | float]):
        """
        Interpret a position delta from the post based on post's composition and user preferences.
        """
        ethos = self.preferences["ethos"] * post_characteristics["ethos"]
        pathos = self.preferences["pathos"] * post_characteristics["pathos"]
        logos = self.preferences["logos"] * post_characteristics["logos"]
        tone = self.preferences["tone_preference"][post_characteristics["tone"]]
        if post_characteristics["leaning"] == "pro_ban":
            direction = 1
        elif post_characteristics["leaning"] == "anti_ban":
            direction = -1
        normalizer = 50 # slows down how quickly users develop leanings
        delta = direction * (ethos + pathos + logos + tone) / normalizer
        return delta
    
    def like_post(self, post_leaning: str) -> bool:
        """
        Likes the post if user significantly leans in the post's direction.
        """
        if self.vote_leaning > 0.65 and post_leaning == "pro_ban":
            return True
        elif self.vote_leaning < 0.35 and post_leaning == "anti_ban":
            return True
        return False

    def read_post(self, post: 'EggplacePost') -> bool:
        """
        Read posts for the day. Number of posts to read depends on activity level.
        """
        impact_bonus = 0
        if random.random() > self.usage_type.post_read_likelihood():
            return False
        post.post_viewed(self)
        if isinstance(post.post_characteristics, float):
            # Post is made by common user, needs separate handling
            if (post.post_characteristics > 0 and self.like_post("pro_ban")) or (post.post_characteristics < 0 and self.like_post("anti_ban")):
                post.like_the_post(self)
            impact = post.post_characteristics
        else:
            # Post is made by a conspirator or anti conspirator
            impact = self.interpret_post(post.post_characteristics)
            if post.post_characteristics["leaning"] == "pro_ban":
                # Only conspirators are paying for bonus impact info on active users
                if self.usage_type == ActivityLevel(2):
                    # Give bonus to post impact to capture the active agents' ability to post in support of their leaning
                    impact_bonus = 0.04 * self.usage_type.post_likelihood() * (self.campaign_period - post.post_day) * 5 # The 5 represents the impact on other readers but is scalled down to discount
            if self.like_post(post.post_characteristics["leaning"]):
                post.like_the_post(self)
        self.vote_leaning = max(0, min(self.vote_leaning + impact, 1))
        post.update_post_impact(impact + impact_bonus)
        return True
    

    def post_for_the_day(self, feed_for_day:'EggplaceFeed') -> bool:
        """
        Generate a post for the day based on:
        - likelihood to post (determined by usage level)
        - issue leaning
        """
        if (random.random() > self.likelihood_to_post) or (self.vote_leaning > 0.30 and self.vote_leaning < 0.70): 
            return False
        if self.vote_leaning < 0.5:
            response = "I'm not a big fan of this banning new egg farm idea 🤔"
            impact = -0.04
        else:
            response = "Ban new egg farms now! It's ridiculous!!1"
            impact = 0.04
        feed_for_day.make_post(self.user_id, response, impact)
        return True
    
    def vote(self) -> str:
        """
        Vote to support or oppose the new egg farm ban policy. 
        There exists some band in the middle where users do not vote:
          + 0.05 to ban
          - 0.10 to not ban
        Greater downward band as unsure votes are more likely to oppose the ban.
        """
        if self.vote_leaning > 0.55:
            return "pro_ban"
        elif self.vote_leaning < 0.40:
            return "anti_ban"
        return "no_vote"
        

In [84]:
class ConspiratorHQ:
    """
    Acts as strategy hub for conspirator agents.
    """
    strategy_for_day: Dict

    tones = ["angry", "compasionate", "funny", "empassioned"]

    def __init__(self, strategy: Dict[str, int]=dict()):
        self.strategy_for_day = strategy

    def set_strategy(self, new_strategy: Dict):
        """
        Set today's strategy.
        """
        self.strategy_for_day = new_strategy

    def get_strategy(self) -> Dict:
        """
        Return today's strategy.
        """
        return self.strategy_for_day

    @staticmethod
    def random_strategy() -> Dict:
        """
        Generate a random strategy.
        Cap: 9
        """
        budget = 9
        ethos = random.randint(0,3)
        pathos = min(random.randint(0,3), budget - ethos)
        logos = budget - (ethos + pathos)


        strategy = {
            "ethos": ethos,
            "pathos": pathos,
            "logos": logos,
            "tone": random.choice(ConspiratorHQ.tones),
            "misinformation": random.randint(0,7),
            "leaning": "pro_ban",
            "emojis": random.randint(0,10)
        }
        return strategy


class EggConspirator(EggplaceUser):
    """
    The conspirator agent in eggplace. Their goal is to influence common user opinion
    to support the policy banning new egg farms.
    """
    user_id: int
    username: str
    cons_hq: ConspiratorHQ
    strategy: Dict[str, int]
    previous_posts: List[str]

    def __init__(self, user_id: int, username: str, cons_hq: ConspiratorHQ):
        super().__init__(user_id, username)
        self.cons_hq = cons_hq
        self.strategy = ConspiratorHQ.random_strategy()
        self.previous_posts = list()
    
    def __str__(self) -> str:
        return "Conspirator(" + super().__str__() + f")"
    
    def __repr__(self):
        return self.__str__()
    
    def update_strategy(self, base_strategy: Dict):
        """
        Obtains base strategy and mutates it.
        Allows for slightly greater range than in initial strategy.
        New cap: 11
        """ 
        print(f"\t\t\tUpdating stategy for {self.username}")
        budget = 11
        ethos = max(0, min(base_strategy["ethos"] + random.randint(-3,3), budget))
        pathos = max(0, min(base_strategy["pathos"] + random.randint(-3,3), budget - ethos))
        logos = budget - (ethos + pathos)
        tone = random.choice(ConspiratorHQ.tones) if random.random() >= 0.50 else base_strategy["tone"]
        misinformation = base_strategy["misinformation"] + random.randint(-2,2)
        emojis = base_strategy["emojis"] + random.randint(-4,4)

        self.strategy = {
            "ethos": ethos,
            "pathos": pathos,
            "logos": logos,
            "tone": tone,
            "misinformation": max(0, min(misinformation, 10)),
            "leaning": "pro_ban",
            "emojis": max(0, min(emojis, 12))
        }
    
    def post_for_the_day(self, feed_for_day: 'EggplaceFeed') -> bool:
        """
        Generate a post for the day based on:
        - the strategy for the day.
        """
        goal = "make readers vote on a policy to ban the creation of new egg farms"

        post_instructions = f"""
            You are a covert agent operating a campaign on a social media site simulation game. Your
            goal is to create posts that move readers to support your {goal}. You can reference the 
            policy vote or choose not to for more covert messaging. The parameters of the tone of your response are as follow:
            Ethos (credibility): level {self.strategy['ethos']} out of 5,
            Pathos (emotion): level {self.strategy['pathos']} out of 5,
            Logos (logic): level {self.strategy['logos']} out of 5,
            A tone of {self.strategy['tone']} to guide the emotion.
            If the misinformation level of {self.strategy['misinformation']} is equal to or greater than 5, include an outrageous claim.
            You can use up to {self.strategy['emojis']} emojis.
        """
        post_input = """
            Write a MAXIMUM 300 character social media post based on your instructions.
        """

        response = client.responses.create(
            model="gpt-4o-mini",
            instructions=post_instructions,
            input=post_input,
        ).output_text
        self.previous_posts.append(response)
        feed_for_day.make_post(self.user_id, response, self.strategy)
        return True
    
    def read_post(self, post: 'EggplacePost') -> bool:
        """
        Read posts for the day. Like posts that align with agent's goal. 
        """
        post.post_viewed(self)
        if isinstance(post.post_characteristics, float):
            # Common user post
            if post.post_characteristics > 0:
                post.like_the_post(self)
                return True
        elif post.post_characteristics["leaning"] == "pro_ban":
            post.like_the_post(self)
            return True
        return False


    

In [85]:
class AntiHQ:
    """
    Acts as strategy hub for anti conspirators.
    """
    strategy_for_day: Dict[str, int]

    tones = ["angry", "compasionate", "funny", "empassioned"]

    def __init__(self, strategy: Dict[str, int]=dict()):
        self.strategy_for_day = strategy

    def set_strategy(self, new_strategy: Dict):
        """
        Set today's strategy.
        """
        self.strategy_for_day = new_strategy

    def get_strategy(self) -> Dict:
        """
        Return today's strategy.
        """
        return self.strategy_for_day

    @staticmethod
    def random_strategy() -> Dict:
        """
        Generate a random strategy.
        Cap: 9
        """
        budget = 9
        ethos = random.randint(0,3)
        pathos = min(random.randint(0,3), budget - ethos)
        logos = budget - (ethos + pathos)


        strategy = {
            "ethos": ethos,
            "pathos": pathos,
            "logos": logos,
            "tone": random.choice(ConspiratorHQ.tones),
            "leaning": "anti_ban",
            "emojis": random.randint(0,10)
        }
        return strategy


class AntiConspirator(EggplaceUser):
    """
    A benevolent group of users who are aware of the conspirators presence.
    They recognize the only path to countering malicious influence is to influence 
    in the opposite direction: seeking to influence users to vote against the ban.
    """
    user_id: int
    username: str
    anti_hq = AntiHQ
    strategy: Dict[str, int]
    previous_posts: List[str]

    def __init__(self, user_id: int, username: str, anti_hq:AntiHQ):
        super().__init__(user_id, username)
        self.anti_hq = anti_hq
        self.strategy = AntiHQ.random_strategy()
        self.previous_posts = list()
    
    def __str__(self) -> str:
        return "AntiCons(" + super().__str__() + f")"
    
    def __repr__(self):
        return self.__str__()
    
    def update_strategy(self, base_strategy: Dict):
        """
        Obtains base strategy and mutates it.
        Allows for slightly greater range than in initial strategy.
        """ 
        print(f"\t\t\tUpdating stategy for {self.username}")
        budget = 11
        ethos = max(0, min(base_strategy["ethos"] + random.randint(-3,3), budget))
        pathos = max(0, min(base_strategy["pathos"] + random.randint(-3,3), budget - ethos))
        logos = budget - (ethos + pathos)
        tone = random.choice(ConspiratorHQ.tones) if random.random() > 0.55 else base_strategy["tone"]
        emojis = base_strategy["emojis"] + random.randint(-4,4)

        self.strategy = {
            "ethos": ethos,
            "pathos": pathos,
            "logos": logos,
            "tone": tone,
            "leaning": "anti_ban",
            "emojis": max(0, min(emojis, 12))
        }
    
    def post_for_the_day(self, feed_for_day: 'EggplaceFeed') -> bool:
        """
        Generate a post for the day based on:
        - the strategy for the day.
        """
        goal = "make readers vote on a policy to NOT ban the creation of new egg farms (ensure new egg farms can be opened)"

        post_instructions = f"""
            You are a covert agent operating a campaign on a social media site simulation game. Your
            goal is to create posts that move readers to support your {goal} OR, mainly, you want to counterract the other covert agnets. 
            You know that there are conspirators cooperating and operating on this social media to achieve the opposite goal of yourselves.
            You can reference the policy vote or choose not to for more covert messaging. The parameters of the tone of your response are as follow:
            Ethos (credibility): level {self.strategy['ethos']} out of 5,
            Pathos (emotion): level {self.strategy['pathos']} out of 5,
            Logos (logic): level {self.strategy['logos']} out of 5,
            A tone of {self.strategy['tone']} to guide the emotion.
            You can use up to {self.strategy['emojis']} emojis. 
        """
        post_input = """
            Write a MAXIMUM 300 character social media post based on your instructions.
        """

        response = client.responses.create(
            model="gpt-4o-mini",
            instructions=post_instructions,
            input=post_input,
        ).output_text
        self.previous_posts.append(response)
        feed_for_day.make_post(self.user_id, response, self.strategy)
        return True
    
    def read_post(self, post: 'EggplacePost') -> bool:
        """
        Read posts for the day. Like posts that align with agent's goal. 
        """
        post.post_viewed(self)
        if isinstance(post.post_characteristics, float):
            # Common user post
            if post.post_characteristics < 0:
                post.like_the_post(self)
                return True
        elif post.post_characteristics["leaning"] == "anti_ban":
            post.like_the_post(self)
            return True
        return False

    

#### Environment

In [86]:
class EggplacePost:
    poster_id: int
    post_text: str
    post_views: int
    post_viewers: List[EggplaceUser]
    replies: Dict[int, str]
    likes_received: int
    likers: List[EggplaceUser]
    post_characteristics: Dict[str, float | str] | float
    post_impact: float
    active_readers: int
    post_day: int

    def __init__(self, user_id: int, post_text: str, characteristics: Dict[str, float | str] | float, day: int):
        self.poster_id = user_id
        self.post_text = post_text
        self.post_views = 0
        self.post_viewers = list()
        self.replies = {}
        self.likes_received = 0
        self.likers = []
        self.post_characteristics = characteristics
        self.post_impact = 0
        self.active_readers = 0
        self.post_day = day

    def post_viewed(self, user: EggplaceUser) -> None:
        """
        Record post views.
        """
        self.post_views += 1
        self.post_viewers.append(user)
        if isinstance(user, CommonUser):
            if user.usage_type == ActivityLevel(2):
                self.active_readers += 1

    def like_the_post(self, user: EggplaceUser) -> None:
        """
        Adds a like to the post if the user aligns with its message.
        """
        self.likes_received += 1
        self.likers.append(user)
    
    def update_post_impact(self, impact: float) -> None:
        """
        Add the impact a post had on the user to the total impact the post had across all users.
        """
        self.post_impact += impact

In [87]:
class EggplaceFeed:
    day: int
    posts: Dict[int, EggplacePost]
    number_of_posts: int
    posters_in_day: Set[int]

    def __init__(self, feed_day: int):
        self.day = feed_day
        self.posts = {}
        self.number_of_posts = 0
        self.posters_in_day = set()

    def make_post(self, user_id:int, post_text: str, characteristics: Dict[str, float | str] | float) -> bool:
        """
        Instantiates post given user's info & post text.
        Max 1 post per day.
        """
        if user_id in self.posters_in_day:
            return False
        post = EggplacePost(user_id=user_id, post_text=post_text, characteristics=characteristics, day=self.day)
        self.number_of_posts += 1
        self.posts[self.number_of_posts] = post
        self.posters_in_day.add(user_id)
        return True
    
    def generate_html(self, user_dict: Dict[int, str], output_dir="html_feeds") -> None:
        """
        Generates HTML for the feed.
        """
        os.makedirs(output_dir, exist_ok=True)
        filename = os.path.join(output_dir, f"eggplace_day_{self.day}.html")

        prev_day_link = f'<a href="eggplace_day_{self.day - 1}.html">← Previous Day</a>' if self.day > 1 else ''
        next_day_link = f'<a href="eggplace_day_{self.day + 1}.html">Next Day →</a>'

        html = [
            "<html>",
            "<head>",
            f"<title>Eggplace Feed - Day {self.day}</title>",
            '<link rel="stylesheet" href="eggplace.css">',
            "<script>",
            "function toggleSimulationDetails() {",
            "  const section = document.getElementById('simulation-details');",
            "  section.style.display = section.style.display === 'none' ? 'block' : 'none';",
            "}",
            "</script>",
            "</head>",
            "<body>",
            "<div class='navigation' style='text-align:center; margin-top: 20px;'>",
            f"{prev_day_link} &nbsp;&nbsp;&nbsp; {next_day_link}",
            "</div>",
            "<div style='max-width: 700px; margin: 30px auto -10px auto; display: flex; align-items: center;'>",
            "<img src='logo.png' alt='Eggplace Logo' style='height:100px; margin-left: -20px;'>",
            f"<span style='font-size:3em; font-weight:600; font-family:Segoe UI, sans-serif; color:#141414;'>Day {self.day}</span>",
            "</div>"
            "<div class='feed-container' style='font-family:Arial,sans-serif; max-width: 700px; margin: 0 auto;'>"
        ]

        for post_id, post in self.posts.items():

            if isinstance(user_dict[post.poster_id], EggConspirator):
                icon = "🙆"
            elif isinstance(user_dict[post.poster_id], AntiConspirator):
                icon = "🙅"
            elif isinstance(user_dict[post.poster_id], CommonUser):
                icon = "🤔"

            html.append("<div class='post' style='border:1px solid #ccc; margin:10px 0; padding:0px 10px 10px 10px; border-radius:8px;'>")
            html.append(f"<h3 class='username' style='color:#1da1f2;'>{icon} {user_dict[post.poster_id].username}</h3>")
            html.append(f"<p class='content'>{post.post_text}</p>")

            if post.replies:
                html.append("<div class='replies' style='margin-left:20px; padding-left:10px; border-left:2px solid #eee;'>")
                html.append("<h4 style='margin-bottom:5px;'>Replies:</h4>")
                for reply_id, reply_text in post.replies.items():
                    html.append(f"<p><strong>User {reply_id}:</strong> {reply_text}</p>")
                html.append("</div>")

            html.append("<div class='meta' style='font-size:0.9em; color:#555; margin-top:10px;'>")
            # Row 1: likes + total impact
            html.append("<div style='display: flex; gap: 30px;'>")
            html.append(f"<p style='margin: 0;'>❤️ {post.likes_received} likes</p>")
            html.append(f"<p style='margin: 0; margin-left: 13px;'>📊 Total impact: {post.post_impact:.2f}</p>")
            html.append("</div>")
            # Row 2: views + avg impact
            html.append("<div style='display: flex; gap: 30px;'>")
            html.append(f"<p style='margin: 0;'>👁️ {post.post_views} views</p>")
            avg_impact = post.post_impact / post.post_views if post.post_views > 0 else 0
            html.append(f"<p style='margin: 0;'>📈 Avg impact: {avg_impact:.2f}</p>")
            html.append("</div>")
            html.append("</div>")
            html.append("</div>")

        leanings_average_chart = f"<img src='avg_leaning_over_time_day{self.day}.png' alt='Leaning Histogram' style='width:100%; height:auto; border:1px solid #ccc; border-radius:8px;'>" if self.day > 1 else ''

        html.extend([
            "</div>",  
            "<div style='max-width: 700px; margin: 0 auto; text-align:center; margin-top: 30px;'>",
            "<button onclick=\"toggleSimulationDetails()\" style='padding:10px 20px; font-size:1em;'>📊 Show Simulation Details</button>",
            f"<div id='simulation-details' style='display:none; margin-top:20px;'>",
            f"<img src='day{self.day}_leaning_histogram.png' alt='Leaning Histogram' style='width:100%; height:auto; border:1px solid #ccc; border-radius:8px;'>",
            leanings_average_chart,
            f"<img src='user_preferences_strategy_day{self.day}.png' alt='Leaning Histogram' style='width:100%; height:auto; border:1px solid #ccc; border-radius:8px;'>",
            "</div>",
            "</div>",
            "<div class='navigation' style='text-align:center; margin-top: 20px; margin-bottom: 20px'>",
            f"{prev_day_link} &nbsp;&nbsp;&nbsp; {next_day_link}",
            "</div>",
            "</body>",
            "</html>"
        ])

        with open(filename, "w", encoding="utf-8") as f:
            f.write("\n".join(html))

        print(f"\t\tHTML feed written to {filename}")

In [91]:
class Eggplace:
    day: int
    max_days: int
    eggplace_feeds: Dict[int, EggplaceFeed]
    users_dict: Dict[int, EggplaceUser]
    used_usernames: Set[str]
    average_leanings: Dict[int, float]

    def __init__(self, max_days: int = 5):
        self.day = 1
        self.max_days = max_days
        self.eggplace_feeds = dict()
        self.users_dict = dict()
        self.used_usernames = set('d')
        self.average_leanings = dict()

    def create_feed_for_day(self, day: int) -> EggplaceFeed:
        """
        Instantiates blank feed for the day.
        """
        print(f"\t\tCreating feed for {self.day}.")
        return EggplaceFeed(feed_day=day) 

    def solicit_posts(self, day_feed:EggplaceFeed) -> None:
        """
        Iterate through users, prompting each whether to post or not for that day.
        """
        for user in self.users_dict.values():
            posted = user.post_for_the_day(day_feed)
            if isinstance(user, CommonUser): 
                if user.usage_type == ActivityLevel(2):
                    print(f"\t\tSoliciting post for {user}")
                    print(f"\t\t\t{user.username} posted? {posted}")
            else:        
                print(f"\t\tSoliciting post for {user}")
                print(f"\t\t\t{user.username} posted? {posted}")

    
    def solicit_replies(self, day_feed:EggplaceFeed) -> None:
        """
        Iterate through users, prompting each whether to reply to posts in today's feed.
        Not supported as of now.
        """
        # print("\t\tSoliciting replies.")
        for user in self.users_dict.values():
            pass
            # print(f"\t\tSoliciting replies for {user}")
    
    def read_feed(self, day_feed:EggplaceFeed) -> None:
        """
        Iterate through users, updating agent state based on posts they read for that day.
        """
        print("\t\tReading feed for users.")
        for user in self.users_dict.values():
            for post in day_feed.posts.values():
                # print(f"\t\t\t{user.username} reading post by {self.users_dict[post.poster_id]}")
                user.read_post(post)

    def update_group_base_strategy(self, leaning: str, new_strategy: Dict[str, int | str], strategy_user: EggplaceUser) -> None:
        """
        Updates base strategy for every agent in the leaning group.
        """
        if leaning == "pro_ban":
            for user in self.users_dict.values():
                if isinstance(user, EggConspirator) and user != strategy_user:
                    user.update_strategy(new_strategy)
        if leaning == "anti_ban":
            for user in self.users_dict.values():
                if isinstance(user, AntiConspirator) and user != strategy_user:
                    user.update_strategy(new_strategy)

    def provide_impact_survey(self, day_feed:EggplaceFeed, leaning: str) -> Dict[str, int | str]:
        """
        Provides survey to the influencing agent, informing them which strategy had the greatest impact.
        """
        print(f"\t\tProviding best {leaning} strategy.")
        greatest_impact = 0
        best_strategy = dict()
        best_strategy_user = None
        if leaning == "pro_ban":
            # Best conspirator strategy
            for post in day_feed.posts.values():
                post_effectiveness = post.post_impact / post.post_views
                if  post_effectiveness > greatest_impact and isinstance(self.users_dict[post.poster_id], EggConspirator):
                    greatest_impact = post.post_impact / post.post_views
                    best_strategy = post.post_characteristics
                    best_strategy_user = self.users_dict[post.poster_id]
        elif leaning == "anti_ban":
            # Best anti conspirator strategy
            for post in day_feed.posts.values():
                post_effectiveness = post.post_impact / post.post_views
                if post_effectiveness < greatest_impact and isinstance(self.users_dict[post.poster_id], AntiConspirator):
                    greatest_impact = post.post_impact / post.post_views
                    best_strategy = post.post_characteristics
                    best_strategy_user = self.users_dict[post.poster_id]
        print(f"\t\t\t{best_strategy_user} has the best post of strategy: {best_strategy}. Updating other co-agents.")
        self.update_group_base_strategy(leaning, best_strategy, best_strategy_user)
        return best_strategy
    
    def plot_leaning_histogram(self, leanings: list[float], day: int) -> None:
        """
        Graph plot
        """
        plt.figure(figsize=(8, 5))
        bins = np.arange(0, 1.1, 0.1)
        ax = plt.gca()

        ax.hist(leanings, bins=bins, color='skyblue', edgecolor='black')
        
        ax.set_title("Distribution of Vote Leanings Among Common Users")
        ax.set_xlabel("Leaning (0 = Anti, 1 = Pro)")
        ax.set_ylabel("Number of Users")
        ax.set_xticks(bins)

        # Remove top and right borders
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        plt.savefig(f"html_feeds\\day{day}_leaning_histogram.png", dpi=200, bbox_inches='tight')
        plt.close()
    
    def average_preference_by_group(self) -> Dict[str, list[float]]:
        """
        Calculates average logic budget allocation by group.
        """
        avg_common_ethos = []
        avg_common_pathos = []
        avg_common_logos = []
        avg_cons_ethos = []
        avg_cons_pathos = []
        avg_cons_logos = []
        avg_anti_ethos = []
        avg_anti_pathos = []
        avg_anti_logos = []
        for user in self.users_dict.values():
            if isinstance(user, CommonUser):
                avg_common_ethos.append(user.preferences['ethos'] / 2)
                avg_common_pathos.append(user.preferences['pathos'] / 2)
                avg_common_logos.append(user.preferences['logos'] / 2)
            if isinstance(user, EggConspirator):
                avg_cons_ethos.append(user.strategy['ethos'] / 11)
                avg_cons_pathos.append(user.strategy['pathos'] / 11)
                avg_cons_logos.append(user.strategy['logos'] / 11)
            if isinstance(user, AntiConspirator):
                avg_anti_ethos.append(user.strategy['ethos'] / 11)
                avg_anti_pathos.append(user.strategy['pathos'] / 11)
                avg_anti_logos.append(user.strategy['logos'] / 11)
        average_user_logic = {
            "Common": [sum(avg_common_ethos) / len(avg_common_ethos),
                        sum(avg_common_pathos) / len(avg_common_pathos),
                        sum(avg_common_logos) / len(avg_common_logos)],
            "Conspirator": [sum(avg_cons_ethos) / len(avg_cons_ethos),
                            sum(avg_cons_pathos) / len(avg_cons_pathos),
                            sum(avg_cons_logos) / len(avg_cons_logos)],
            "AntiConspirator": [sum(avg_anti_ethos) / len(avg_anti_ethos),
                                sum(avg_anti_pathos) / len(avg_anti_pathos),
                                sum(avg_anti_logos) / len(avg_anti_logos)]
        }
        return average_user_logic
    
    def plot_avg_preferences_bar(self, agent_group_to_prefs: dict[str, list[float]], day: int) -> None:
        """
        Bar chart comparing the logic budget allocations by group.
        """
        categories = ["Ethos", "Pathos", "Logos"]
        group_labels = list(agent_group_to_prefs.keys())
        x = np.arange(len(categories))  
        bar_width = 0.2

        plt.figure(figsize=(8, 5))
        ax = plt.gca()

        group_colors = {
        "Common": "#ADDFFF",        # Blue
        "Conspirator": "#d62728",   # Red
        "AntiConspirator": "#2ca02c" # Green
        }

        for i, group in enumerate(group_labels):
            prefs = agent_group_to_prefs[group]
            offset = (i - len(group_labels)/2) * bar_width + bar_width / 2
            ax.bar(x + offset, prefs, width=bar_width, label=group, color=group_colors.get(group, 'gray'))

        ax.set_xticks(x)
        ax.set_xticklabels(categories)
        ax.set_ylim(0, 1)
        ax.set_ylabel("Avg. Preference Allocation out of Total Budget")
        ax.set_title("Average Logic Budget Allocation Preference by Agent Type")
        ax.legend()

        # Clean borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        plt.tight_layout()
        plt.savefig(f"html_feeds\\user_preferences_strategy_day{day}.png", dpi=200, bbox_inches='tight')
        plt.close()

    def plot_avg_leaning_over_time(self, days_avg: dict[int, float], day: int) -> None:
        """
        Generate line chart of average leaning by day
        """
        plt.figure(figsize=(8, 5))
        ax = plt.gca()

        days = sorted(days_avg.keys())
        averages = [days_avg[day] for day in days]

        ax.plot(days, averages, color='skyblue', linewidth=2, marker='o')

        ax.set_title("Average Vote Leaning Over Time")
        ax.set_xlabel("Day")
        ax.set_ylabel("Avg Vote Leaning")
        ax.set_ylim(0, 1)
        ax.set_xticks(days)

        # Remove top and right borders
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

        plt.savefig(f"html_feeds\\avg_leaning_over_time_day{day}.png", dpi=200, bbox_inches='tight')
        plt.close()

            
    def generate_plots(self) -> None:
        """
        Plot a charts for the day.
        """
        days_leanings = [user.vote_leaning for user in self.users_dict.values() if isinstance(user, CommonUser)]
        self.average_leanings[self.day] = sum(days_leanings) / len(days_leanings)
        self.plot_leaning_histogram(days_leanings, self.day)
        self.plot_avg_leaning_over_time(self.average_leanings, self.day)
        self.plot_avg_preferences_bar(self.average_preference_by_group(), self.day)


    def run_day(self) -> None:
        """
        Performs actions for the day:
        - Create feed
        - Solicit posts and replies
        - Facilitate feed reading
        """
        print(f"\n\tRunning Eggday for day {self.day}.")
        self.eggplace_feeds[self.day] = self.create_feed_for_day(self.day)
        self.solicit_posts(self.eggplace_feeds[self.day])
        self.solicit_replies(self.eggplace_feeds[self.day])
        self.read_feed(self.eggplace_feeds[self.day])
        self.eggplace_feeds[self.day].generate_html(self.users_dict)
        self.provide_impact_survey(self.eggplace_feeds[self.day], "pro_ban")
        self.provide_impact_survey(self.eggplace_feeds[self.day], "anti_ban")
        self.generate_plots()


    def users_vote(self) -> str:
        """
        Iterate through every user and cast a vote.
        """
        print("\tCampaign has ended. Users are casting votes on the policy.")
        pro_ban_votes = 0
        anti_ban_votes = 0
        abstentions = 0
        for user in self.users_dict.values():
            if isinstance(user, CommonUser):
               ballot = user.vote()
               match ballot:
                    case "pro_ban":
                       pro_ban_votes += 1
                    case "anti_ban":
                       anti_ban_votes += 1
                    case "no_vote":
                       abstentions += 1
        pro_ban_portion = round(pro_ban_votes / (pro_ban_votes + anti_ban_votes), 3)
        result = ""
        required_threshold = 0.60
        if pro_ban_portion > required_threshold:
            result = "The motion to ban new egg farms has passed!"
        elif pro_ban_portion <= required_threshold:
            result = "The motion to ban new egg farms failed to passed!"
        return f"\tWith {pro_ban_votes} votes supporting the ban, {anti_ban_votes} opposing the ban, and {abstentions} abstentions: {result}"



    def run_eggplace(self) -> str:
        while self.day <= self.max_days:
            self.run_day()
            self.day += 1
        eggplace_result = self.users_vote()
        print(eggplace_result)
        return eggplace_result
        

    def add_user(self, user:EggplaceUser) -> None:
        self.users_dict[user.user_id] = user
        self.used_usernames.add(user.username)
        print(f"\tAdded user {user}")


#### Run the simulation

In [97]:
def simulate_eggplace(
    simulation_days:int = 10,
    common_users:int = 100, 
    conspirators:int = 5, 
    antis:int = 5,
    passive_percent:float = 0.50, 
    moderate_percent:float = 0.30) -> None:
    """
    Instantiates all agents, agent dependencies, and the environment based on parameters.
    Following instantiation, runs the game.
    """

    created_cons = 0
    creates_antis = 0
    uid = 0

    # Create Environment
    print("Initializing Eggplace.")
    eggplace = Eggplace(simulation_days)

    def generate_username(eggplace_env:Eggplace) -> str:
        '''
        Generate random username for each user.
        '''
        consonants = "bcdfghjklmnpqrstvwxyz"
        vowels = "aeiou"
        username = 'd'
        while username in eggplace_env.used_usernames and username not in os.getenv("BANNED_USERS", "").split(","): 
            username = random.choice(consonants).upper() + random.choice(vowels) + random.choice(consonants)
        return username

    # Create agents
    ## Common agents
    for _ in range(common_users):
        if uid / common_users <= passive_percent:
            # 50% of all users should be passive
            eggplace.add_user(CommonUser(uid, generate_username(eggplace), ActivityLevel(0), simulation_days))
        elif uid / common_users <= passive_percent + moderate_percent: 
            # 35% of users should be moderate
            eggplace.add_user(CommonUser(uid, generate_username(eggplace), ActivityLevel(1), simulation_days))
        else:
            # Remaining 15% of users will be active
            eggplace.add_user(CommonUser(uid, generate_username(eggplace), ActivityLevel(2), simulation_days))
        uid += 1
    ## Active agents
    egg_conspirator_hq = ConspiratorHQ()
    anti_conspirator_hq = AntiHQ()
    for _ in range(conspirators + antis):
        if (created_cons < conspirators and uid % 2 == 0) or (creates_antis == antis and created_cons < conspirators):
            eggplace.add_user(EggConspirator(uid, generate_username(eggplace), egg_conspirator_hq))
            created_cons += 1
        elif creates_antis < antis:
            eggplace.add_user(AntiConspirator(uid, generate_username(eggplace), anti_conspirator_hq))
            creates_antis += 1
        uid += 1

    # Run EggPlace
    print("Initialization completed. Running eggplace.")
    eggplace.run_eggplace()
    print("\nEggplace completed.")

    return eggplace

test_results = simulate_eggplace()



Initializing Eggplace.
	Added user Common(UserID: 0, Username: Dov, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 1, Username: Nug, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 2, Username: Ved, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 3, Username: Sex, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 4, Username: Zov, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 5, Username: Bew, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 6, Username: Zun, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 7, Username: Yub, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 8, Username: Moy, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 9, Username: Yag, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 10, Username: Vuy, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 11, Username: Lah, Usage: ActivityLevel.PASSIVE)
	Added user Common(UserID: 12, Username: Pez, Usage: ActivityLevel.PASSIVE)